In [24]:
import os
import shutil
import random
from pathlib import Path

import numpy as np
from sklearn.metrics import classification_report
from torchvision import datasets, models, transforms
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [25]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Task 1

In [30]:
train_dir = "/kaggle/working/train"
val_dir = "/kaggle/working/val"
test_dir = "/kaggle/working/test"
classes = ["duck", "chicken"]

In [31]:
# Create directories
for split in [train_dir, val_dir, test_dir]:
    for cls in classes:
        os.makedirs(os.path.join(split, cls), exist_ok=True)

In [32]:
# Split data: 70% train, 15% val, 15% test
split_ratio = [0.8, 0.1, 0.1]
for cls in classes:
    src_folder = os.path.join("/kaggle/working", cls)
    images = os.listdir(src_folder)
    random.shuffle(images)
    n_total = len(images)
    n_train = int(n_total * split_ratio[0])
    n_val = int(n_total * split_ratio[1])

    for i, img in enumerate(images):
        if i < n_train:
            dst = os.path.join(train_dir, cls, img)
        elif i < n_train + n_val:
            dst = os.path.join(val_dir, cls, img)
        else:
            dst = os.path.join(test_dir, cls, img)
        shutil.copy(os.path.join(src_folder, img), dst)

In [33]:
# Define transformations
transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [34]:
# Load datasets
image_datasets = {
    x: datasets.ImageFolder(os.path.join("/kaggle/working", x), transform[x])
    for x in ['train', 'val', 'test']
}

In [35]:
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2)
    for x in ['train', 'val', 'test']
}

In [36]:
class_names = image_datasets['train'].classes

In [37]:
# Load pre-trained model and modify
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: duck & chicken
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [38]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Training loop
def train_model(model, criterion, optimizer, num_epochs=5):
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model

In [40]:
model = train_model(model, criterion, optimizer, num_epochs=10)


Epoch 1/10
----------
train Loss: 0.6884 Acc: 0.7704
val Loss: 5.0959 Acc: 0.8113

Epoch 2/10
----------
train Loss: 0.3737 Acc: 0.8396
val Loss: 0.4962 Acc: 0.8302

Epoch 3/10
----------
train Loss: 0.1967 Acc: 0.9245
val Loss: 0.4596 Acc: 0.8113

Epoch 4/10
----------
train Loss: 0.1929 Acc: 0.9214
val Loss: 0.3253 Acc: 0.8679

Epoch 5/10
----------
train Loss: 0.2593 Acc: 0.8868
val Loss: 0.3957 Acc: 0.8491

Epoch 6/10
----------
train Loss: 0.1624 Acc: 0.9403
val Loss: 0.1530 Acc: 0.9057

Epoch 7/10
----------
train Loss: 0.2332 Acc: 0.8994
val Loss: 0.2733 Acc: 0.9245

Epoch 8/10
----------
train Loss: 0.1365 Acc: 0.9465
val Loss: 0.2089 Acc: 0.9245

Epoch 9/10
----------
train Loss: 0.0559 Acc: 0.9780
val Loss: 0.0737 Acc: 0.9434

Epoch 10/10
----------
train Loss: 0.0429 Acc: 0.9811
val Loss: 0.1105 Acc: 0.9811


In [41]:
# Evaluation
model.eval()
y_true, y_pred = [], []


In [42]:
with torch.no_grad():
    for inputs, labels in dataloaders["test"]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

In [43]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


Classification Report:
              precision    recall  f1-score   support

     chicken       0.96      0.93      0.95        28
        duck       0.93      0.96      0.95        28

    accuracy                           0.95        56
   macro avg       0.95      0.95      0.95        56
weighted avg       0.95      0.95      0.95        56

